In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ML').getOrCreate()

21/08/15 20:00:41 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.86.23 instead (on interface wlo1)
21/08/15 20:00:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/08/15 20:00:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
training = spark.read.csv('data/test1.csv', header=True, inferSchema=True)
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [4]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [5]:
training.columns

['Name', 'age', 'Experience', 'Salary']

[Age, Experience] =====> new feature ----> independent feature

In [7]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=['age', 'Experience'], outputCol='Independent Features')

In [8]:
output = featureassembler.transform(training)
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [9]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [10]:
finalized_data = output.select("Independent Features", "Salary")

In [11]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [12]:
from pyspark.ml.regression import LinearRegression
## train test split
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor = regressor.fit(train_data)

21/08/15 20:09:48 WARN Instrumentation: [ea2e0ffe] regParam is zero, which might cause numerical instability and overfitting.
21/08/15 20:09:48 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
21/08/15 20:09:48 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
21/08/15 20:09:48 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
21/08/15 20:09:48 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [13]:
regressor.coefficients

DenseVector([-263.7076, 1767.624])

In [14]:
regressor.intercept

19919.060052212404

In [15]:
pred_result = regressor.evaluate(test_data)

In [16]:
pred_result.predictions.show()

+--------------------+------+-----------------+
|Independent Features|Salary|       prediction|
+--------------------+------+-----------------+
|          [29.0,4.0]| 20000|19342.03655352618|
+--------------------+------+-----------------+



In [17]:
pred_result.meanAbsoluteError, pred_result.meanSquaredError

(657.9634464738192, 432915.89689570636)